In [9]:
import modern_robotics as mr
import numpy as np
import sympy as sp

from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

from Utilities.symbolicFunctions import *

In [10]:
# #Funksjoner som trengs

# def PsFromTsd(T_sd):
#     #Finner Ps fra T_sd
#     #T_sd gir konfigurasjonen vi vil ha end-effector framen, B, i.
#     #B, og derav også M, er lik som i DH
#     #s er plassert nederst på roboten med positiv z oppover, altså ikke som i DH. Bør kanskje endres til å være lik DH 
#     P_d = np.array([0,0,80,1])
#     P_s = T_sd@P_d

#     return P_s

# def exp3(omega, theta):
#     omega = skew(omega)
#     R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
#     return R

# def skew(v):
#     return Matrix([[0, -v[2], v[1]],
#                     [v[2], 0, -v[0]],
#                     [-v[1], v[0], 0]])
                    
# def exp6(twist, theta):
#     omega = skew(twist[:3])
#     v = Matrix(twist[3:])
#     T = eye(4)
#     T[:3,:3] = exp3(twist[:3], theta)
#     T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
#               (theta-sin(theta)) * omega * omega) * v
#     return T


# def rotX(alfa_im1):
#     Rx = sp.eye(4)
#     Rx[1,1] =    sp.cos(alfa_im1)
#     Rx[1,2] =   -sp.sin(alfa_im1)
#     Rx[2,1] =    sp.sin(alfa_im1)
#     Rx[2,2] =    sp.cos(alfa_im1)
#     return Rx

# def rotZ(alfa_im1):
#     Rz = sp.eye(4)
#     Rz[0,0] =    sp.cos(alfa_im1)
#     Rz[0,1] =   -sp.sin(alfa_im1)
#     Rz[1,0] =    sp.sin(alfa_im1)
#     Rz[1,1] =    sp.cos(alfa_im1)
#     return Rz

# def rotY(alfa_im1):
#     Ry = sp.eye(4)
#     Ry[0,0] =    sp.cos(alfa_im1)
#     Ry[0,2] =    sp.sin(alfa_im1)
#     Ry[2,0] =   -sp.sin(alfa_im1)
#     Ry[2,2] =    sp.cos(alfa_im1)
#     return Ry




In [11]:
#Definerer S og M
"""
S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,455+25])
S4 = np.array([-1,0,0,0,-435,0])
S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
print(Slist)

M = np.array([[0,0,-1,455+25+420+80],
              [0,1,0,0],
              [1,0,0,400],
              [0,0,0,1]])
"""
def Slist_maker(omega_mat, q_mat): #omega_mat and q_mat of type matrix with q_i & omega_i as columns
    #Returns v_mat in same type/format
    v_mat = calc_v(omega_mat, q_mat)    
    n_joints = omega_mat.shape[1]
    Slist = sp.zeros(6, n_joints)
    
    for i in range(n_joints):
        Slist[:3,i] = omega_mat[:,i]
        Slist[3:,i] = v_mat[:,i]
    return Slist

def calc_v(omega_mat, q_mat):
    #omega_mat and q_mat of type matrix with q_i & omega_i as columns
    #Returns v_mat in same type/format
    assert len(omega_mat) == len(q_mat)
    
    n_joints = omega_mat.shape[1] 
    v_mat = sp.zeros(3, n_joints)      

    for i in range(n_joints):
        v_mat[:,i] = (-skew(omega_mat.col(i)) * q_mat.col(i))
    return v_mat



M1=sp.Matrix([[0, 1, 0, 0],
             [1, 0, 0, 0],
             [0, 0, -1, 200],
             [0, 0, 0, 1]])

M2=sp.Matrix([[1, 0, 0, 25],
             [0, 0, 1, 0],
             [0, -1, 0, 400],
             [0, 0, 0, 1]])

M3=sp.Matrix([[1, 0, 0, 455+25],
             [0, 0, 1, 0],
             [0, -1, 0, 400],
             [0, 0, 0, 1]])
M4=sp.Matrix([[0, 0, -1, 455+25+200], #420
             [0, 1, 0, 0],
             [1, 0, 0, 400+35],
             [0, 0, 0, 1]])
M5=sp.Matrix([[1, 0, 0, 455+25+420],
             [0, 0, 1, 0],
             [0, -1, 0, 400+35],
             [0, 0, 0, 1]])
M6=sp.Matrix([[0, 0, -1, 455+25+420+80], ##OBS lagt til 50 for å se endeffector
             [0, 1, 0, 0],
             [1, 0, 0, 400+35],
             [0, 0, 0, 1]])
Mlist = np.array([M1,M2,M3,M4,M5,M6], dtype=float)

om = sp.zeros(3,6)
om1 = om[:, 0] = M1[:3, 2]
om2 = om[:, 1] = M2[:3, 2]
om3 = om[:, 2] = M3[:3, 2]
om4 = om[:, 3] = M4[:3, 2]
om5 = om[:, 4] = M5[:3, 2]
om6 = om[:, 5] = M6[:3, 2]
q = sp.zeros(3,6)
q1 = q[:,0] = M1[:3, 3]
q2 = q[:,1] = M2[:3, 3]
q3 = q[:,2] = M3[:3, 3]
q4 = q[:,3] = M4[:3, 3]
q5 = q[:,4] = M5[:3, 3]
q6 = q[:,5] = M6[:3, 3]

Slist = np.array(Slist_maker(om,q),dtype=float)

M = np.array(M6,dtype=float)

thetas_up = [0,0,0,0,0,0]
thetas_down = [0,0,0,0,0,0]

print(Slist)
#Limits til roboten slik den er gitt i oppgaven. Antar at ledd 5 har limit på +-90


[[   0.    0.    0.   -1.    0.   -1.]
 [   0.    1.    1.    0.    1.    0.]
 [  -1.    0.    0.    0.    0.    0.]
 [   0. -400. -400.    0. -435.    0.]
 [   0.    0.    0. -435.    0. -435.]
 [   0.   25.  480.    0.  900.    0.]]


In [12]:
from Utilities.symbolicFunctions import *

In [13]:

#OPPGAVE 3.2

#Her endres thetas_gen for å teste forskjellige konfigurasjoner:
thetas_gen = np.array([2,1,4,2,1,2])
T_sd = mr.FKinSpace(M,Slist,thetas_gen)
print("T_sd\n", T_sd)

P_s = PsFromTsd(T_sd)
print("Ps", P_s)

#theta1
thetas_up[0] = -sp.atan2(P_s[1],P_s[0]) #minus foran fordi vinkelen er definert andre vei
thetas_down[0] = thetas_up[0]

#theta2 and theta3
a = np.sqrt(420**2+35**2)
c = 455
P_merket = np.array([P_s[0],P_s[1],P_s[2]-400])
b = np.sqrt((np.sqrt(P_merket[0]**2+P_merket[1]**2)-25)**2 + P_merket[2]**2)#Merk: -25 pga offset i skulder

psi = np.arccos(420/a) #Vinkelen mellom den faktiske armen og den vi tegna for å få en trekant(Pga 35mm offset i elbow). Se notatbok
phi = sp.atan2(P_merket[2], sp.sqrt(P_merket[0]**2 + P_merket[1]**2)-25)

alpha = np.arccos((b**2+c**2-a**2)/(2*b*c)) #Vil ha negativ alpha når b krysser offesettet, som i null-posisjonen
beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

print("alpha:", alpha, "beta:", beta, "phi",phi, psi, "b:", b)

#thetas_up[2] = np.pi/2 - (beta-psi)
#thetas_down[2] =  -(2*np.pi - (beta+psi) - np.pi/2)
thetas_down[1] =  -(phi-alpha)
thetas_up[2] = np.pi - beta + psi
if(thetas_up[2]<0):
    alpha = -alpha
#thetas_down[2] =  -thetas_up[2]
thetas_down[2] =  -(np.pi - beta - psi)

#thetas_up[1] = np.pi/2 - (alpha + phi) #Index 1 på thetas[1] og thetas[2] gir elbow down løsning
thetas_up[1] = -(alpha + phi)
#thetas_down[1] =  np.pi/2 - (phi-alpha)


print(thetas_up, thetas_down)

#Vi har XYX euler angles. De er egentlig (-X)Y(-X) fordi det er slik S'ene er definert,.

#Elbow down:
T1 = exp6(Slist[:,0], -thetas_down[0])
T2 = exp6(Slist[:,1], -thetas_down[1])
T3 = exp6(Slist[:,2], -thetas_down[2])

R = (T3@T2@T1@T_sd@np.linalg.inv(M)) #R er den resterende rotasjonen vi ønsker å få fra de tre siste leddene, definert i s

thetas_down[3] = -sp.atan2(R[1,0], -R[2,0]) #minus foran theta4 og 6 fordi de er i minus x retning
thetas_down[4] = sp.atan2(sp.sqrt(1-R[0,0]**2), R[0,0])
thetas_down[5] = -sp.atan2(R[0,1], R[0,2])


#Elbow up:
T1 = exp6(Slist[:,0], -thetas_up[0])
T2 = exp6(Slist[:,1], -thetas_up[1])
T3 = exp6(Slist[:,2], -thetas_up[2])

R = (T3@T2@T1@T_sd@np.linalg.inv(M))

thetas_up[3] = -sp.atan2(R[1,0], -R[2,0])
thetas_up[4] = sp.atan2(sp.sqrt(1-R[0,0]**2), R[0,0])
thetas_up[5] = -sp.atan2(R[0,1], R[0,2])

T_sd
 [[-7.8126e-01  7.4168e-02  6.1979e-01 -1.9790e+02]
 [-3.0648e-01  8.1941e-01 -4.8439e-01 -2.8533e+02]
 [-5.4379e-01 -5.6839e-01 -6.1744e-01  4.7920e+02]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
Ps [-148.3203 -324.0857  429.8071    1.    ]
alpha: 1.0896979375776463 beta: 0.7752661145217655 phi 0.0896979375776459 0.08314123188844062 b: 332.75103099820745
[-1.14159265358979 + pi, -1.17939587515529, 2.4494677709564683, 0, 0, 0] [-1.14159265358979 + pi, 1.00000000000000, -2.2831853071795867, 0, 0, 0]


In [14]:
#testing av analytisk løsning:

#UP
thetas_up_num = np.zeros(6)
thetas_up_deg = np.zeros(6)
for i in range(0,6):
    thetas_up_num[i] = sp.N(thetas_up[i])
    thetas_up_deg[i] = np.rad2deg(thetas_up_num[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

T_up_thetas = mr.FKinSpace(M,Slist,thetas_up_num)
P_up_reached = PsFromTsd(T_up_thetas)
#print(P_reached, P_s)
np.set_printoptions(precision=4)

#DOWN
thetas_down_num = np.zeros(6)
thetas_down_deg = np.zeros(6)
for i in range(0,6):
    thetas_down_num[i] = sp.N(thetas_down[i])
    thetas_down_deg[i] = np.rad2deg(thetas_down_num[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

T_down_thetas = mr.FKinSpace(M,Slist,thetas_down_num)
P_down_reached = PsFromTsd(T_down_thetas)


#fk_test = exp6(S4,thetas_num[3])@exp6(S5,thetas_num[4])@exp6(S6,thetas_num[5])@M
#R_test = rotX(thetas_num[3])@rotY(thetas_num[4])@rotX(thetas_num[5]) 
thetas_calc, asd = mr.IKinSpace(Slist,M,T_sd,[0,0,0,0,0,0],0.01,0.01)

T_sd, thetas_gen, T_up_thetas, thetas_up_num, T_down_thetas, thetas_down_num, P_s, P_up_reached, P_down_reached

(array([[-7.8126e-01,  7.4168e-02,  6.1979e-01, -1.9790e+02],
        [-3.0648e-01,  8.1941e-01, -4.8439e-01, -2.8533e+02],
        [-5.4379e-01, -5.6839e-01, -6.1744e-01,  4.7920e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([2, 1, 4, 2, 1, 2]),
 array([[-7.8126e-01,  7.4168e-02,  6.1979e-01, -1.9790e+02],
        [-3.0648e-01,  8.1941e-01, -4.8439e-01, -2.8533e+02],
        [-5.4379e-01, -5.6839e-01, -6.1744e-01,  4.7920e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([ 2.    , -1.1794,  2.4495,  1.582 ,  2.2702, -0.4215]),
 array([[-7.8126e-01,  7.4168e-02,  6.1979e-01, -1.9790e+02],
        [-3.0648e-01,  8.1941e-01, -4.8439e-01, -2.8533e+02],
        [-5.4379e-01, -5.6839e-01, -6.1744e-01,  4.7920e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([ 2.    ,  1.    , -2.2832,  2.    ,  1.    ,  2.    ]),
 array([-148.3203, -324.0857,  429.8071,    1.    ]),
 array([-148.3203, -324.0857,  429.8071,

In [15]:
T_sd, thetas_up_num, thetas_down_num

jointLimits = np.array([[-180, 180], [-190, 45], [-120, 156], [-180, 180], [-90, 90], [-180, 180]]) #Assuming joint 5 has limits [-90, 90]

def applyJointLim(jointLimits, thetas):
    ''' Check if Inverse Kinematics solution (thetas) is within jointlimits\n 
    PARAMETERS:
    Jointlimits, numpy 2D array with lower and upper limits in deg\n
    Jointangles, numpy 1D array in rad
    RETURNS: Boolean true or false, if false a print message with the offending link is printed '''
    jointLimits = np.deg2rad(jointLimits) 
    thetas %= 2*np.pi #Post processing: all thetas in [0,2*pi)

    for theta, i in enumerate(thetas):
        if jointLimits[i][1] < theta < jointLimits[i][0]:
            print("Joint number: ", i+1, "is not within the limits")
            return False
    return True


In [16]:
from Utilities.RobotClass import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [17]:
Kuka = Robot(Mlist, ['z', '-z', 'x', 'x', '-z','x'])

In [18]:
Kuka.draw_robot()
Kuka.transform(Slist, thetas_down_num ) # Elbow DOWN
pUp = Kuka.joints[5].coord.get_center()
Kuka.draw_robot()
Kuka.transform(Slist, thetas_up_num ) # Elbow UP
pDown = Kuka.joints[5].coord.get_center()
Kuka.draw_robot()


WebVisualizer(window_uid='window_0')

WebVisualizer(window_uid='window_1')

WebVisualizer(window_uid='window_2')

In [19]:
print(pUp, pDown)

[-198.0161 -285.2979  476.9679] [-198.0161 -285.2979  476.9679]
